In [60]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import os
import time
import pandas as pd
import numpy as np
import re
import nltk
nltk.download('vader_lexicon')
class SeleniumClient(object):
    def __init__(self):
        #Initialization method. 
        self.chrome_options = webdriver.ChromeOptions()
        self.chrome_options.add_argument('--headless')
        self.chrome_options.add_argument('--no-sandbox')
        self.chrome_options.add_argument('--disable-setuid-sandbox')
        self.chrome_options.add_argument("start-maximized")
        self.chrome_options.add_argument("disable-infobars")
        self.chrome_options.add_argument("--disable-extensions")

        # you need to provide the path of chromdriver in your system
    

        chromedriver = "C:/Users/ManasiAdmin/chromedriver"
        os.environ["webdriver.chrome.driver"] = chromedriver
       
        
        self.driver = webdriver.Chrome(chromedriver)
        #driver.get("https://twitter.com/search?q=amex")
        #time.sleep(10)
        #driver.quit()

        
        #self.browser = webdriver.Chrome('C:/Users/ManasiAdmin/chromedriver', options=self.chrome_options)

        self.base_url = 'https://twitter.com/search?q='

    def get_tweets(self, query):
        ''' 
        Function to fetch tweets. 
        '''
        #try: 
        self.driver.get(self.base_url+query)
        time.sleep(10)

        body = self.driver.find_element_by_tag_name('body')

        for _ in range(500):
            body.send_keys(Keys.PAGE_DOWN)
            time.sleep(0.3)

        timeline = self.driver.find_element_by_id('timeline')
        #user_nodes = timeline.find_element_by_css_selector("h1.ProfileHeaderCard-name a.ProfileHeaderCard-nameLink")
        tweet_nodes = timeline.find_elements_by_css_selector('.tweet-text')
            #tweet_nodes = timeline.find_elements_by_css_selector('[id^=\".tweet-text\"]')

        return pd.DataFrame({'tweets': [tweet_node.text for tweet_node in tweet_nodes] ,'tweet_id': [tweet_node.id for tweet_node in tweet_nodes]})

        #except: 
                #print("Selenium - An error occured while fetching tweets.")

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\ManasiAdmin\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [64]:
selenium_client = SeleniumClient()

tweets_df = selenium_client.get_tweets('visa card')

In [65]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from textblob import TextBlob 
def fetch_sentiment_using_SIA(text):
    sid = SentimentIntensityAnalyzer()
    polarity_scores = sid.polarity_scores(text)
    if polarity_scores['neg'] > polarity_scores['pos']:
        return 'negative'
    else:
        return 'positive'

def fetch_sentiment_using_textblob(text):
    analysis = TextBlob(text)
    # set sentiment 
    if analysis.sentiment.polarity >= 0:
        return 'positive'
    else: 
        return 'negative'

In [66]:
sentiments_using_textblob = tweets_df.tweets.apply(lambda tweet: fetch_sentiment_using_textblob(tweet))
pd.DataFrame(sentiments_using_textblob.value_counts())

,tweets
positive,608
negative,84


In [67]:
sentiments_using_SIA = tweets_df.tweets.apply(lambda tweet: fetch_sentiment_using_SIA(tweet))
pd.DataFrame(sentiments_using_SIA.value_counts())

,tweets
positive,567
negative,125


In [68]:
#We get less biased results using textblob
tweets_df['sentiment'] = sentiments_using_textblob
tweets_df

,tweets,tweet_id,sentiment
0,The @FirstBankngr Visa Brand has a Card for ev...,0.6925770837198724-3,positive
1,You will think Nigerians don’t like better thi...,0.6925770837198724-4,positive
2,The Beat: $500 Visa Prepaid Gift Card Sweepsta...,0.6925770837198724-5,positive
3,Galaxy + Ghoul Trooper Account!\n\nFULL ACCESS...,0.6925770837198724-6,positive
4,"@Office365 Hey guys, I have a small problem, I...",0.6925770837198724-7,negative
5,"Barefaced as you are, without shame, you added...",0.6925770837198724-8,positive
6,Anyone who can solve my riddle here in this vi...,0.6925770837198724-9,positive
7,Nimble Visa Card | Nimble Visa Prepaid Card be...,0.6925770837198724-10,positive
8,Get a $100 Visa Gift Card\nhttps://onlinesoffe...,0.6925770837198724-11,positive
9,#Contest: Win A $50 Visa Gift Card In April Fr...,0.6925770837198724-12,positive


In [69]:
def remove_pattern(text, pattern_regex):
    r = re.findall(pattern_regex, text)
    for i in r:
        text = re.sub(i, '', text)
    
    return text
# We are keeping cleaned tweets in a new column called 'tidy_tweets'
tweets_df['tidy_tweets'] = np.vectorize(remove_pattern)(tweets_df['tweets'], "@[\w]*: | *RT*")

In [70]:
cleaned_tweets = []

for index, row in tweets_df.iterrows():
    # Here we are filtering out all the words that contains link
    words_without_links = [word for word in row.tidy_tweets.split()        if 'http' not in word]
    cleaned_tweets.append(' '.join(words_without_links))

tweets_df['tidy_tweets'] = cleaned_tweets

In [71]:
tweets_df.drop_duplicates(subset=['tidy_tweets'], keep=False)

,tweets,tweet_id,sentiment,tidy_tweets
0,The @FirstBankngr Visa Brand has a Card for ev...,0.6925770837198724-3,positive,The @FirstBankngr Visa Brand has a Card for ev...
1,You will think Nigerians don’t like better thi...,0.6925770837198724-4,positive,You will think Nigerians don’t like better thi...
2,The Beat: $500 Visa Prepaid Gift Card Sweepsta...,0.6925770837198724-5,positive,The Beat: $500 Visa Prepaid Gift Card Sweepsta...
3,Galaxy + Ghoul Trooper Account!\n\nFULL ACCESS...,0.6925770837198724-6,positive,Galaxy + Ghoul Trooper Account! FULL ACCESS (M...
4,"@Office365 Hey guys, I have a small problem, I...",0.6925770837198724-7,negative,"@Office365 Hey guys, I have a small problem, I..."
5,"Barefaced as you are, without shame, you added...",0.6925770837198724-8,positive,"Barefaced as you are, without shame, you added..."
6,Anyone who can solve my riddle here in this vi...,0.6925770837198724-9,positive,Anyone who can solve my riddle here in this vi...
7,Nimble Visa Card | Nimble Visa Prepaid Card be...,0.6925770837198724-10,positive,Nimble Visa Card | Nimble Visa Prepaid Card be...
8,Get a $100 Visa Gift Card\nhttps://onlinesoffe...,0.6925770837198724-11,positive,Get a $100 Visa Gift Card …
9,#Contest: Win A $50 Visa Gift Card In April Fr...,0.6925770837198724-12,positive,#Contest: Win A $50 Visa Gift Card In April Fr...


In [72]:
#Extracting those id's where sentiment is negative (dissatisfied)
tweets_df.loc[tweets_df['sentiment'] == 'negative']

,tweets,tweet_id,sentiment,tidy_tweets
4,"@Office365 Hey guys, I have a small problem, I...",0.6925770837198724-7,negative,"@Office365 Hey guys, I have a small problem, I..."
25,"It's almost 5 pm, time to catch up with the fr...",0.6925770837198724-28,negative,"It's almost 5 pm, time to catch up with the fr..."
62,Platinum and Gold Visa Card Pizza and Ice Crea...,0.6925770837198724-65,negative,Platinum and Gold Visa Card Pizza and Ice Crea...
89,@ICICIBank_Care Hi I am unable to use your poc...,0.6925770837198724-92,negative,@ICICIBank_Care Hi I am unable to use your poc...
111,"@KimIversenShow Hi Kim, you may hate twitter b...",0.6925770837198724-114,negative,"@KimIversenShow Hi Kim, you may hate twitter b..."
121,Don’t be a victim of Visa card frauds. In case...,0.6925770837198724-124,negative,Don’t be a victim of Visa card frauds. In case...
129,LIKE = VISA CARD WITH 2.3$ IN YOUR DM\n\nFOLLO...,0.6925770837198724-132,negative,LIKE = VISA CAD WITH 2.3$ IN YOU DM FOLLOW @Po...
133,Do you have the Uber-branded Visa card? Are yo...,0.6925770837198724-136,negative,Do you have the Uber-branded Visa card? Are yo...
143,"On Feb 18, I was trying to buy plane tickets f...",0.6925770837198724-146,negative,"On Feb 18, I was trying to buy plane tickets f..."
156,Having Real Time Gross Settlement Dollars (RTG...,0.6925770837198724-159,negative,Havingeal Time Gross Settlement Dollars (GS $)...


In [73]:
#Removing punctuations numbers and special characters
tweets_df['absolute_tidy_tweets'] = tweets_df['tidy_tweets'].str.replace("[^a-zA-Z# ]", "")

In [74]:
#Removing stopwords
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords_set = set(stopwords.words("english"))
cleaned_tweets = []

for index, row in tweets_df.iterrows():
    
    # filerting out all the stopwords 
    words_without_stopwords = [word for word in row.absolute_tidy_tweets.split() if not word in stopwords_set]
    
    # finally creating tweets list of tuples containing stopwords(list) and sentimentType 
    cleaned_tweets.append(' '.join(words_without_stopwords))

tweets_df['absolute_tidy_tweets'] = cleaned_tweets

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ManasiAdmin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [75]:
from nltk.stem import WordNetLemmatizer
# Tokenization
tokenized_tweet = tweets_df['absolute_tidy_tweets'].apply(lambda x: x.split())
# Finding Lemma for each word
word_lemmatizer = WordNetLemmatizer()
tokenized_tweet = tokenized_tweet.apply(lambda x: [word_lemmatizer.lemmatize(i) for i in x])
#joining words into sentences (from where they came from)
for i, tokens in enumerate(tokenized_tweet):
    tokenized_tweet[i] = ' '.join(tokens)

tweets_df['absolute_tidy_tweets'] = tokenized_tweet

In [76]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
# BOW features
bow_word_vectorizer = CountVectorizer(max_df=0.90, min_df=2, stop_words='english')
# bag-of-words feature matrix
bow_word_feature = bow_word_vectorizer.fit_transform(tweets_df['absolute_tidy_tweets'])

# TF-IDF features
tfidf_word_vectorizer = TfidfVectorizer(max_df=0.90, min_df=2, stop_words='english')
# TF-IDF feature matrix
tfidf_word_feature = tfidf_word_vectorizer.fit_transform(tweets_df['absolute_tidy_tweets'])

In [77]:
target_variable = tweets_df['sentiment'].apply(lambda x: 0 if x=='negative' else 1 )

In [78]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
def naive_model(X_train, X_test, y_train, y_test):
    naive_classifier = GaussianNB()
    naive_classifier.fit(X_train.toarray(), y_train)

    # predictions over test set
    predictions = naive_classifier.predict(X_test.toarray())
    
    # calculating f1 score
    print(f'F1 Score - {f1_score(y_test, predictions)}')

In [79]:
X_train, X_test, y_train, y_test = train_test_split(bow_word_feature, target_variable, test_size=0.3, random_state=870)
naive_model(X_train, X_test, y_train, y_test)

F1 Score - 0.8983957219251336
